In [ ]:
# passporteye for reading mrz 
!pip install passporteye

# reading images 
!pip install opencv-python

# encrypt 
!pip install cryptography 



In [19]:
from PIL import Image

# Load a JPG image
img = Image.open(r'C:\Users\ethan\mnt\data\K.jpg')


# Convert to RGB 
rgb_img = img.convert('RGB')

# Save it as a PNG
rgb_img.save(r'C:\Users\ethan\mnt\data\imageKP.png', 'PNG')


In [ ]:
# passportEye Code
# read_mrz() from the passporteye library
# 
from passporteye import read_mrz

# image of the passport located at the
mrz = read_mrz(r'C:\Users\ethan\mnt\data\imageKP.png')

if mrz is not None:
    print("MRZ Data Found!")
    print(mrz)
    print("\nParsed Data:")
    print(mrz.to_dict())
    data = mrz.to_dict()
else:
    print("No MRZ found in the image.")
    
# Opens or create folder
with open (r'C:\Users\ethan\mnt\results\mrz_KPassportData2.txt', 'w') as f:
        f.write("MRZ Parsed Data:\n\n")
        for key, value in data.items():
            f.write(f"{key}: {value}\n") 
print("MRZ data saved in results folder!!")

#if mrz.valid_score and mrz.valid_score > 50:
#    print("MRZ is likely valid")
#else:
#    print("Low MRZ confidence. Might be incorrect.")


In [ ]:
# update code passporteyecode 1.0
# this code will load an image of a passport
# extracts MRZ data from it
# covents that data into a dictionary
# saves the results into a .txt file 
# it will tell you if the results is trustworthy or not 
# still in testing lol !!!
import os
from passporteye import read_mrz
from PIL import Image

# Input and output paths
input_path = r'C:\Users\Owner\mnt\data\imageKP.png'
output_path = r'C:\Users\Owner\mnt\results\mrz_KPassportData.txt'

#
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Read MRZ
mrz = read_mrz(input_path)

if mrz is not None:
    print("MRZ Data Found!")
    print(mrz)
    print("\nParsed Data:")
    data = mrz.to_dict()
    print(data)

    # Save to text file
    with open(output_path, 'w') as f:
        f.write("MRZ Parsed Data:\n\n")
        for key, value in data.items():
            f.write(f"{key}: {value}\n")

    print(f"\n MRZ data saved to: {output_path}")

    if mrz.valid_score and mrz.valid_score > 50:
        print("MRZ is likely valid!!!")
    else:
        print("Low MRZ confidence. Results may be inaccurate!!!")

else:
    print("No MRZ found in the image SORRY. Nothing was saved.")



In [ ]:
import cv2
import pytesseract as ract
from passporteye import read_mrz
import numpy as np

ract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Preprocess the photo for better OCR 
def preprocessed_photo(photo_path):
    photo = cv2.imread(photo_path)

    # Resize for better detail
    photo = cv2.resize(photo, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)

    # Convert to grayscale
    gray = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)

    # Denoise
    gray = cv2.fastNlMeansDenoising(gray, h=31)

    thresh = cv2.adaptiveThreshold(
        gray, 254, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY, 30, 14
    )

    # Save preprocessed photo
    temp_path = r"C:\Users\Owner\Python MRZ\preprocessed passports\preprocessed_passporteditversion.png"
    cv2.imwrite(temp_path, thresh)

    return temp_path

# Run MRZ Extraction 
photo_path = r'C:\Users\Owner\Python MRZ\data\imageEthanP.png'  
preprocessed_photo = preprocessed_photo(photo_path)
mrz = read_mrz(preprocessed_photo)

# calcue the score for passport
def mrz_score(data_dict):
    necessary_fields = [
        'surname', 'names', 'number', 'date_of_birth', 'expiration_date',
        'nationality', 'sex', 'personal_number'
    ]
    # MRZ integrity flags
    total = len(necessary_fields)
    correct = 0

    for field in necessary_fields:
        value = data_dict.get(field, "")
        if value and value != "<<<<<<<<<<<<<<<<":
            correct += 1

    score = int((correct / total) * 100)
    return score

# Handle Results 
if mrz is not None:
    print("MRZ Data Found!")
    data = mrz.to_dict()

    print("\nParsed Data:")
    print(data)

    # calculte the score and prints score 
    score = mrz_score(data)
    print(f"\nExtraction Score: {score}%")

    # Save to text file
    output_path = r'C:\Users\Owner\Python MRZ\results\mrz_KPassportEdit.txt'
    with open(output_path, 'w') as f:
        f.write("MRZ-Parsed Data:\n\n")
        for key, value in data.items():
            f.write(f"{key}: {value}\n")
    # print result  !!
    print(f"MRZ data saved to: {output_path}")

    # Check confidence score
    if mrz.valid_score and mrz.valid_score > 50:
        print("MRZ is valid!!")
    else:
        print("Low MRZ confidence. Results may be inaccurate! - Please redo !")

else:
    print("No MRZ found in the photo!!!")
